## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

pubdate  source  \
0  2025-10-23-16-54-38 +0000  nypost   
1  2025-10-23-16-53-25 +0000     nyt   
2  2025-10-23-16-53-20 +0000     bbc   
3  2025-10-23-16-51-29 +0000     nyt   
4  2025-10-23-16-50-19 +0000    wapo   

                                               title  \
0  NY AG Letitia James heads to Virginia for mort...   
1         Can Ken Burns Win the American Revolution?   
2  Ghana's former First Lady Nana Konadu Rawlings...   
3  U.S. May Seek United Nations Mandate for Gaza ...   
4            Trump pardons convicted Binance founder   

                                                link  
0  https://nypost.com/2025/10/23/us-news/ny-ag-le...  
1  https://www.nytimes.com/2025/10/23/arts/televi...  
2  https://www.bbc.com/news/articles/c62ex80zp2no...  
3  https://www.nytimes.com/2025/10/23/world/middl...  
4  https://www.washingtonpost.com/business/2025/1...

## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2372/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


word  score
36       trump     54
59         new     26
282  sanctions     15
238    russian     14
284     russia     12

## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


pubdate  source  \
328  2025-10-22-20-58-00 +0000     wsj   
244  2025-10-23-03-10-21 +0000  nypost   
136  2025-10-23-11-45-10 +0000     bbc   
101  2025-10-23-13-45-34 +0000    wapo   
106  2025-10-23-13-32-00 +0000     nyt   

                                                 title  \
328  President Trump has privately told allies he b...   
244  Entire White House East Wing will be demolishe...   
136  Trump says Putin talks 'don't go anywhere' as ...   
101  Russia faces surprise setback with Trump’s new...   
106  Trump Imposes Sanctions on Russian Oil Compani...   

                                                  link  score  
328  https://www.wsj.com/politics/elections/trump-z...    143  
244  https://nypost.com/2025/10/22/us-news/entire-w...    136  
136  https://www.bbc.com/news/articles/cd6758pn6ylo...    124  
101  https://www.washingtonpost.com/world/2025/10/2...    123  
106  https://www.nytimes.com/2025/10/22/us/politics...    114

## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


score                    pubdate  source  \
328    143  2025-10-22-20-58-00 +0000     wsj   
292     70  2025-10-22-22-47-41 +0000    wapo   
244     56  2025-10-23-03-10-21 +0000  nypost   
248     51  2025-10-23-02-41-32 +0000  nypost   
263     49  2025-10-23-01-23-33 +0000  nypost   
305     47  2025-10-22-22-08-46 +0000  nypost   
347     46  2025-10-22-20-16-45 +0000  nypost   
370     43  2025-10-22-18-47-38 +0000  nypost   
104     42  2025-10-23-13-34-07 +0000  nypost   
13      38  2025-10-23-16-31-00 +0000     wsj   

                                                 title  \
328  President Trump has privately told allies he b...   
292  U.S. imposes sanctions on Russian oil companie...   
244  Entire White House East Wing will be demolishe...   
248  Mom of 4 shot dead by ‘obsessed’ ex-boyfriend ...   
263  Andrew Cuomo bolts for Knicks opener minutes a...   
305  Elon Musk’s Tesla disappoints investors despit...   
347  GM embraces AI with plans to use ‘eyes-off’ dr...   
370  US military bombs 8th, 9th suspected drug-traf...   
104  King Charles, Pope Leo pray together in rare m...   
13   Helicopter Crash Kills Four Family Members of ...   

                                                  link  
328  https://www.wsj.com/politics/elections/trump-z...  
292  https://www.washingtonpost.com/world/2025/10/2...  
244  https://nypost.com/2025/10/22/us-news/entire-w...  
248  https://nypost.com/2025/10/22/world-news/young...  
263  https://nypost.com/2025/10/22/us-news/andrew-c...  
305  https://nypost.com/2025/10/22/business/elon-mu...  
347  https://nypost.com/2025/10/22/business/gm-embr...  
370  https://nypost.com/2025/10/22/us-news/us-milit...  
104  https://nypost.com/2025/10/23/world-news/king-...  
13   https://www.wsj.com/us-news/montana-helicopter...

## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
